# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd



# Read the CSV file
tic_tac_toe = pd.read_csv('tic-tac-toe.csv')

# Inspect the dataset
tic_tac_toe.isnull().sum() # Data has no missing values
tic_tac_toe.describe() # Data structure looks correct
tic_tac_toe.head() # Data looks reliable


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [2]:
from sklearn.preprocessing import LabelEncoder

# Convert the categorical values to numeric in all columns
le = LabelEncoder()
for column in tic_tac_toe.columns:
    if tic_tac_toe[column].dtype == 'object':
        tic_tac_toe[column] = le.fit_transform(tic_tac_toe[column])
        
# Separate the inputs and output
tic_tac_toe_columns = tic_tac_toe.columns
predictors = tic_tac_toe[tic_tac_toe_columns[tic_tac_toe_columns != 'class']] # inputs (all columns except class)
target = tic_tac_toe['class'] # output (class column)

# Normalize the input data
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()
n_cols = predictors_norm.shape[1] # number of predictors

predictors.head()
print(predictors.head())

target.head()
print(target.head())

   TL  TM  TR  ML  MM  MR  BL  BM  BR
0   2   2   2   2   1   1   2   1   1
1   2   2   2   2   1   1   1   2   1
2   2   2   2   2   1   1   1   1   2
3   2   2   2   2   1   1   1   0   0
4   2   2   2   2   1   1   0   1   0
0    True
1    True
2    True
3    True
4    True
Name: class, dtype: bool


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [64]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf  
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense


# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)


# define regression model
def regression_model():
    # Create a Sequential model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # output layer with 2 neurons and softmax activation
    
    # compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
# build the model
model = regression_model()

# fit the model
model.fit(X_train, y_train, epochs=100, verbose=2)

# Evaluate the model with the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.2f}')

# Save the model
model.save('/Users/diegoalonso/Documents/Ironhack/Labs/Week_3/Day_1/lab-neural-networks/your-code/tic-tac-toe.model.keras')



Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 0s - 21ms/step - accuracy: 0.6164 - loss: 0.6502
Epoch 2/100
21/21 - 0s - 1ms/step - accuracy: 0.6493 - loss: 0.6082
Epoch 3/100
21/21 - 0s - 3ms/step - accuracy: 0.6731 - loss: 0.5855
Epoch 4/100
21/21 - 0s - 915us/step - accuracy: 0.7134 - loss: 0.5668
Epoch 5/100
21/21 - 0s - 948us/step - accuracy: 0.7328 - loss: 0.5468
Epoch 6/100
21/21 - 0s - 862us/step - accuracy: 0.7448 - loss: 0.5298
Epoch 7/100
21/21 - 0s - 959us/step - accuracy: 0.7761 - loss: 0.5101
Epoch 8/100
21/21 - 0s - 904us/step - accuracy: 0.7896 - loss: 0.4923
Epoch 9/100
21/21 - 0s - 827us/step - accuracy: 0.8194 - loss: 0.4732
Epoch 10/100
21/21 - 0s - 875us/step - accuracy: 0.8224 - loss: 0.4541
Epoch 11/100
21/21 - 0s - 887us/step - accuracy: 0.8358 - loss: 0.4342
Epoch 12/100
21/21 - 0s - 869us/step - accuracy: 0.8388 - loss: 0.4172
Epoch 13/100
21/21 - 0s - 819us/step - accuracy: 0.8478 - loss: 0.4013
Epoch 14/100
21/21 - 0s - 815us/step - accuracy: 0.8537 - loss: 0.3836
Epoch 15/100
21/21 - 0s - 951us/

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [4]:
# Make predictions on the test set
y_pred = model.predict(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [95]:
# Adjusted Regression Model
from keras.layers import Dense, Dropout

def regression_model():
    # Create a Sequential model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dropout(0.2))  # dropout layer
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))  # dropout layer
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))  # dropout layer
    model.add(Dense(2, activation='softmax'))  # output layer with 2 neurons and softmax activation
    
    # compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00015)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
# build the model
model = regression_model()

# fit the model
model.fit(X_train, y_train, epochs=250, verbose=2)

# Evaluate the model with the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.2f}')

Epoch 1/250


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 1s - 31ms/step - accuracy: 0.5373 - loss: 0.7121
Epoch 2/250
21/21 - 0s - 1ms/step - accuracy: 0.6448 - loss: 0.6480
Epoch 3/250
21/21 - 0s - 1ms/step - accuracy: 0.6463 - loss: 0.6307
Epoch 4/250
21/21 - 0s - 1ms/step - accuracy: 0.6552 - loss: 0.6156
Epoch 5/250
21/21 - 0s - 1ms/step - accuracy: 0.6851 - loss: 0.5849
Epoch 6/250
21/21 - 0s - 1ms/step - accuracy: 0.7015 - loss: 0.5864
Epoch 7/250
21/21 - 0s - 993us/step - accuracy: 0.7045 - loss: 0.5804
Epoch 8/250
21/21 - 0s - 996us/step - accuracy: 0.6985 - loss: 0.5715
Epoch 9/250
21/21 - 0s - 935us/step - accuracy: 0.7299 - loss: 0.5510
Epoch 10/250
21/21 - 0s - 976us/step - accuracy: 0.7313 - loss: 0.5569
Epoch 11/250
21/21 - 0s - 932us/step - accuracy: 0.7194 - loss: 0.5387
Epoch 12/250
21/21 - 0s - 986us/step - accuracy: 0.7418 - loss: 0.5265
Epoch 13/250
21/21 - 0s - 959us/step - accuracy: 0.7522 - loss: 0.5107
Epoch 14/250
21/21 - 0s - 948us/step - accuracy: 0.7597 - loss: 0.4926
Epoch 15/250
21/21 - 0s - 950us/step -

**Which approach(es) did you find helpful to improve your model performance?**

In [6]:
# In the previous model, the accuracy was 0.965. The adjusted regression model is 0.97. and the test loss was 0.16 in the original model vs 0.08 in the adjusted version.
# To improve the model performance I added 2 adittional layers to the model and the learning rate and the number of epochs rate adjusted several times but I was getting varying scores for both accuracy and loss.
# Then I implemented dropout to the layers to stabilizae the results and it improved the model accuracy from 0.94 to 0.97 and the loss from 0.16 to 0.08 and over all it increased the stability of the model.